In [10]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import io
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [11]:
# se cargan las imagenes a la memoria para la red neuronal
dirname = os.path.join(os.getcwd(), '/home/carlos/Programing/ProyectoIA/data') #Cambiar directorio al directorio donde se tiene las carpetas con la data 
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /home/carlos/Programing/ProyectoIA/data/
/home/carlos/Programing/ProyectoIA/data/Señales/SeñalTerremoto2 1
/home/carlos/Programing/ProyectoIA/data/Señales/SeñalTerremoto3 994
/home/carlos/Programing/ProyectoIA/data/Señales/SeñaleSocorroGenero1 992
/home/carlos/Programing/ProyectoIA/data/Señales/SeñalTerremoto4 965
/home/carlos/Programing/ProyectoIA/data/Señales/SeñalSocorroGenero2 303
/home/carlos/Programing/ProyectoIA/data/Señales/SeñalTerremoto1 1339
Directorios leidos: 6
Imagenes en cada directorio [995, 992, 965, 303, 1339, 1072]
suma Total de imagenes en subdirs: 5666


In [12]:
## etiquetado de imágenes para la creación de red neuronal ##

labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
senales=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    senales.append(name[len(name)-1])
    indice=indice+1
 
y = np.array(labels)
X = np.array(images) #convierto de lista a pandas

Cantidad etiquetas creadas:  5666
0 SeñalTerremoto2
1 SeñalTerremoto3
2 SeñaleSocorroGenero1
3 SeñalTerremoto4
4 SeñalSocorroGenero2
5 SeñalTerremoto1


ValueError: setting an array element with a sequence.

In [17]:
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

##Creación de grupos de entrenamiento

#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('grupo de entrenamiento : ', train_X.shape, train_Y.shape)
print('grupo de testeo: ', test_X.shape, test_Y.shape)
 

##normalización de los datos  
#train_X = train_X.astype("float32")
#test_X = test_X.astype("float32")
train_X = train_X / 255.
test_X = test_X / 255.
 
# los labels se transforman en variables categoricas
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

 
#muestra el cambio de categoria a codificación
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])
 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)
 
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Total number of outputs :  6
Output classes :  [0 1 2 3 4 5]
grupo de entrenamiento :  (4532,) (4532,)
grupo de testeo:  (1134,) (1134,)


ValueError: setting an array element with a sequence.

array([array([[[0.76078431, 0.75686275, 0.7372549 ],
        [0.76470588, 0.76078431, 0.74117647],
        [0.76862745, 0.76470588, 0.74509804],
        ...,
        [0.81960784, 0.80784314, 0.78823529],
        [0.82352941, 0.81176471, 0.79215686],
        [0.81960784, 0.80784314, 0.78823529]],

       [[0.76470588, 0.76078431, 0.74117647],
        [0.76470588, 0.76078431, 0.74117647],
        [0.76862745, 0.76470588, 0.74509804],
        ...,
        [0.81176471, 0.80784314, 0.78823529],
        [0.82352941, 0.81176471, 0.79215686],
        [0.81176471, 0.80784314, 0.78823529]],

       [[0.76862745, 0.76470588, 0.74509804],
        [0.76862745, 0.76470588, 0.74509804],
        [0.77254902, 0.76862745, 0.74901961],
        ...,
        [0.80784314, 0.81176471, 0.78823529],
        [0.81568627, 0.81176471, 0.79215686],
        [0.80784314, 0.81176471, 0.78823529]],

       ...,

       [[0.76078431, 0.76078431, 0.72941176],
        [0.76078431, 0.76078431, 0.72941176],
        [0.7686

In [14]:
INIT_LR = 1e-3
epochs = 6
batch_size = 64
 
hands_model = Sequential()
hands_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
hands_model.add(LeakyReLU(alpha=0.1))
hands_model.add(MaxPooling2D((2, 2),padding='same'))
hands_model.add(Dropout(0.5))
 
hands_model.add(Flatten())
hands_model.add(Dense(32, activation='linear'))
hands_model.add(LeakyReLU(alpha=0.1))
hands_model.add(Dropout(0.5)) 
hands_model.add(Dense(nClasses, activation='softmax'))
 
hands_model.summary()
 
hands_model.compile(loss='categorical_crossentropy', optimizer='adagrad',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 21, 28, 32)        896       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 21, 28, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 11, 14, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 4928)              0         
                                                                 
 dense_2 (Dense)             (None, 32)                157728    
                                                      

In [8]:
hands_model_o = hands_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1)
 

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).